# Feature Engineering

In [1]:
# Basics Importation 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas_profiling
import numpy as np
import re
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew #for some statistics
from scipy.stats import skew 
from scipy.stats import boxcox_normmax
from scipy.stats import pearsonr
%matplotlib inline
plt.style.use('ggplot')

# For Detailed Stats Output
import statsmodels.api as sm
# The linear regression models 
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, LassoCV, ElasticNetCV
# Instantiating the linear regression models
ols = LinearRegression()
ridge = Ridge()
lasso  = Lasso()
lassocv = LassoCV()
elasticnet = ElasticNet()
elasticnetcv = ElasticNetCV()
# The tree model
from sklearn import tree
# Instantiating the tree model (regression type)
regressor = tree.DecisionTreeRegressor()
# The ensemble model for random forest and bagging
from sklearn import ensemble
# Instantiating the ensemble models
randomForest = ensemble.RandomForestRegressor()
bagging      = ensemble.BaggingRegressor()
# Instantiating the boost models
gbm          = ensemble.GradientBoostingRegressor()
abr          = ensemble.AdaBoostRegressor()
# xg boost
import xgboost as xgb
# lg boost
import lightgbm as lgb
# K mean clustering
from sklearn.cluster import KMeans
kmeans = KMeans()
# The model selection for cross validation, k fold splits, train_test_split, grid search etc. 
import sklearn.model_selection as ms
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
# Some automatic feature selection functions (recursive finding, best feature selection etc.)
import sklearn.feature_selection as fs
# Importing the different error evaluation/metrics
from sklearn import metrics
from sklearn.metrics import silhouette_score

# Making it so that we can see all columns of the dataframe
pd.set_option('display.max_columns', None)

# Import data
df_train=pd.read_csv('standardized_dummified_df_train_for_neighborhood_clean.csv',index_col='Id')
df_test=pd.read_csv('standardized_dummified_df_test_for_neighborhood_clean.csv',index_col='Id')
df_total=pd.read_csv('standardized_dummified_df_total_for_neighborhood_clean.csv',index_col='Id')
df=df_train

In [2]:
nominal_var_processed=['MSZoning','Street','Alley','LotShape','LandContour','LotConfig','LandSlope',\
             'Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl',\
             'Exterior1st','Exterior2nd','MasVnrType','Foundation',\
             'BsmtFinType1','Heating','CentralAir',\
             'Electrical','Functional','GarageType','GarageFinish',\
             'PavedDrive','Fence','MiscFeature','SaleType','SaleCondition','MSSubClass',\
             'WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MoSold']
ordinal_var_processed=['OverallQual','OverallCond','BedroomAbvGr','Fireplaces',\
             'YrSold','HeatingQC','KitchenQual','ExterQual','ExterCond','BsmtQual',\
             'BsmtCond','BsmtExposure','FireplaceQu','GarageQual','GarageCond','TotalBaths']
cont_var_processed=['LotFrontage','LotArea','YearBuilt','YearRemodAdd','MasVnrArea',\
          'GarageYrBlt','TotalSF','CarCapacity']
cont_var_for_tuning=ordinal_var_processed+cont_var_processed

# Neighborhood - Introducing Interaction Parameter

In [3]:
neighborhood_cols=df_train.columns[df_train.columns.str.contains('Neighborhood__')]
df_train.loc[:,neighborhood_cols].transform(lambda x: np.array(x)*np.array(df_train.loc[:,'TotalSF']))

neighborhood_cols=df_test.columns[df_test.columns.str.contains('Neighborhood__')]
df_test.loc[:,neighborhood_cols].transform(lambda x: np.array(x)*np.array(df_test.loc[:,'TotalSF']))

neighborhood_cols=df_total.columns[df_total.columns.str.contains('Neighborhood__')]
df_total.loc[:,neighborhood_cols].transform(lambda x: np.array(x)*np.array(df_total.loc[:,'TotalSF']))
df_total.sample(10)

,GrLivArea,GarageType__CarPort,PavedDrive__P,SaleCondition__Normal,GarageYrBlt,TotalSF,Neighborhood__Edwards,Neighborhood__NWAmes,Foundation__Wood,Neighborhood__NAmes,YearBuilt,Alley__Pave,Neighborhood__BrDale,SaleType__CWD,EnclosedPorch__1,OpenPorchSF__1,LotConfig__FR3,Exterior2nd__Wd Sdng,BsmtCond,MSSubClass__50,MSSubClass__190,Neighborhood__CollgCr,MSSubClass__120,MSZoning__RM,ExterCond,Condition1__Norm,LotShape__IR2,LotFrontage,ExterQual,MSZoning__RH,Condition1__RRNn,Condition2__PosN,RoofStyle__Mansard,MasVnrArea,Alley__No Alley,LotConfig__FR2,MSSubClass__45,CarCapacity,SaleType__ConLw,RoofMatl__Tar&Grv,Condition2__Norm,Neighborhood__NoRidge,WoodDeckSF__1,MasVnrType__BrkFace,Neighborhood__SawyerW,Exterior1st__HdBoard,OverallCond,MSSubClass__30,HouseStyle__2.5Unf,Functional__Min1,Condition1__Feedr,Foundation__Stone,GarageFinish__Unf,3SsnPorch__1,MSSubClass__80,Neighborhood__StoneBr,Exterior2nd__AsphShn,LotShape__Reg,Neighborhood__BrkSide,GarageType__BuiltIn,RoofStyle__Shed,Neighborhood__SWISU,Exterior2nd__CmentBd,Street__Pave,Functional__Sev,Fence__No Fence,Exterior2nd__Brk Cmn,OverallQual,Exterior1st__Plywood,Exterior1st__VinylSd,RoofStyle__Gable,Exterior2nd__MetalSd,MoSold__7,SaleType__Oth,LotShape__IR3,Exterior2nd__Stucco,CentralAir__Y,HeatingQC,MSSubClass__90,RoofStyle__Hip,MoSold__3,BsmtQual,LotArea,Condition1__RRAn,Exterior2nd__BrkFace,Exterior2nd__Wd Shng,Foundation__PConc,Functional__Maj2,Exterior1st__CemntBd,YrSold,SaleCondition__AdjLand,BldgType__Twnhs,MSSubClass__180,GarageType__Attchd,Neighborhood__ClearCr,MoSold__5,Exterior2nd__VinylSd,Exterior2nd__Stone,Neighborhood__NridgHt,HouseStyle__2Story,RoofMatl__WdShake,SaleType__WD,MSSubClass__85,Exterior1st__WdShing,Condition1__PosA,Condition2__Feedr,Exterior1st__Wd Sdng,Neighborhood__Veenker,Electrical__SBrkr,Functional__Mod,HouseStyle__SLvl,MasVnrType__Stone,MoSold__12,RoofStyle__Gambrel,SaleCondition__Alloca,Neighborhood__Gilbert,Heating__Wall,HouseStyle__SFoyer,Functional__Typ,MoSold__6,GarageQual,LotConfig__CulDSac,BsmtFinType1__Rec,Exterior1st__BrkFace,Exterior2nd__ImStucc,Heating__Grav,Exterior2nd__HdBoard,Electrical__FuseF,MSSubClass__60,MSSubClass__160,MoSold__10,TotalBaths,TotRmsAbvGrd,SaleType__Con,MSSubClass__75,GarageType__No Garage,MoSold__4,BldgType__Duplex,Exterior2nd__CBlock,Foundation__CBlock,Neighborhood__Sawyer,YearRemodAdd,Neighborhood__OldTown,PavedDrive__Y,Exterior1st__BrkComm,GarageType__Basment,Fireplaces,Fence__MnPrv,MasVnrType__None,BldgType__TwnhsE,LandSlope__Mod,GarageFinish__RFn,Exterior1st__Stucco,BsmtFinType1__No Basement,HouseStyle__1.5Unf,SaleType__ConLI,MoSold__11,LandContour__Lvl,Neighborhood__MeadowV,LandSlope__Sev,Condition1__RRNe,BsmtFinType1__BLQ,Condition1__PosN,Functional__Min2,Electrical__FuseP,MSSubClass__70,BsmtFinType1__Unf,SaleType__New,HouseStyle__1Story,Neighborhood__Timber,BsmtFinType1__GLQ,BsmtExposure,GarageCond,Neighborhood__NPkVill,SaleCondition__Partial,Exterior2nd__Plywood,BedroomAbvGr,Exterior1st__AsphShn,KitchenQual,LandContour__HLS,Exterior1st__MetalSd,Heating__GasW,Neighborhood__Mitchel,BldgType__2fmCon,Condition1__RRAe,Exterior1st__CBlock,GarageFinish__No Garage,MasVnrType__No Masonry Veneer,BsmtFinType1__LwQ,LotConfig__Inside,Foundation__Slab,MSZoning__FV,MoSold__8,Neighborhood__Somerst,RoofMatl__WdShngl,MSZoning__RL,LandContour__Low,FireplaceQu,GarageType__Detchd,Neighborhood__Crawfor,Fence__GdWo,ScreenPorch__1,MoSold__9,Neighborhood__Blueste,Fence__MnWw,SaleType__ConLD,MoSold__2,Condition2__PosA,MiscFeature__Shed,MSSubClass__40,Neighborhood__IDOTRR,SaleCondition__Family,PoolArea__1,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2274,-0.292202,0,0,1,0.036141,0.089171,0,1,0,0,0.254450,0,0,0,0,1,0,0,-0.009719,0,0,0,0,0,-0.230171,1,0,0.498192,-0.683591,0,0,0,0,0.854014,1,0,0,-0.032188,0,0,1,0,1,1,0,0,-0.507295,0,0,0,0,0,0,0,0,0,0,1

In [4]:
def undummify(dataframe):
    tot_col=dataframe.columns
    cat_col=list(tot_col[tot_col.str.contains('__')])
    cat_col_split=set(map(lambda x:x.split('__')[0],cat_col))
    cat_dict={}
    for col in cat_col_split:
        sub_df=dataframe[cat_col].loc[:,list(map(lambda x:col in x, dataframe[cat_col].columns))]
        for i in sub_df.columns:
            label_num=int(i.split('__')[1])
            sub_df.loc[:,i]=np.array(sub_df.loc[:,i])*label_num
        cat_dict[col]=sub_df.sum(axis=1)+1
    df1=dataframe.drop(cat_col,axis=1)
    df2=pd.DataFrame(cat_dict)
    return pd.concat([df1,df2],axis=1)

In [5]:
df_train.to_csv('standardized_dummified_df_train_neighborhood_interact.csv')
df_test.to_csv('standardized_dummified_df_test_neighborhood_interact.csv')
df_total.to_csv('standardized_dummified_df_total_neighborhood_interact.csv')